### Khai báo các thư viện

In [1]:
# datetime trả về cả thời gian và ngày tháng
# timedelta trả về khoảng thời gian
from datetime import datetime, timedelta
# thêm thư viện Pandas xử lý database
import pandas as pd
# thêm các thư viện hỗ trợ trực quan hóa data matplotlib, seaborn
import matplotlib.pyplot as plt
#thêm module pylot
import numpy as np
# thêm numpy để sử lý các mảng dữ liệu
import seaborn as sns
from __future__ import division
#thay đổi chế độ chia tương thích phiên bản python hiện tại
import chart_studio.plotly as py
import plotly.offline as pyoff
# sử dụng plotly ở chế độ offline
import plotly.graph_objs as go
# graph_objs vẽ biểu đồ cột

### Import data

In [2]:
#đặt plotly trong trong môi trường notebook
pyoff.init_notebook_mode()
tx_data = pd.read_csv('data.csv')
# đọc data bằng bằng pandas 
tx_data.head(10)
#hiển thị 10 row đầu tiên 

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5    536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6    536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7    536366     22633               HAND WARMER UNION JACK         6   
8    536366     22632            HAND WARMER RED POLKA DOT         6   
9    536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
5  12/1/2010 8:26       7.65     17850.0  United Kingdom  
6  12/1/2010 8:26       4.25     17850.0  United Kingdom  
7  12/1/2010 8:28       1.85     17850.0  United Kingdom  
8  12/1/2010 8:28       1.85     17850.0  United Kingdom  
9  12/1/2010 8:34       1.69     13047.0  United Kingdom

Tính tổng doanh thu mỗi tháng

In [3]:
# định dạnh field invoicedata sang datatime
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])

#tạo field InvoiceYearMonth bằng cách gộp từng ngày của từng tháng tạo doanh thu theo tháng
tx_data['InvoiceYearMonth'] = tx_data['InvoiceDate'].map(lambda date: 100*date.year + date.month)

#Tính doanh thu của từng tháng bằng cách tạo field Revenue
# doanh thu = giá mặt hàng * số lượng
tx_data['Revenue'] = tx_data['UnitPrice'] * tx_data['Quantity']
# gộp những row giống nhau trong Field InvoiceYearMonth 
tx_revenue =tx_data.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
# tính tổng doanh thu của tháng
tx_revenue

InvoiceYearMonth      Revenue
0             201012   748957.020
1             201101   560000.260
2             201102   498062.650
3             201103   683267.080
4             201104   493207.121
5             201105   723333.510
6             201106   691123.120
7             201107   681300.111
8             201108   682680.510
9             201109  1019687.622
10            201110  1070704.670
11            201111  1461756.250
12            201112   433668.010

In [4]:
#trực quan hóa doanh thu dưới dạng biểu đồ
plot_data =[
    go.Scatter(
        x=tx_revenue['InvoiceYearMonth'],
        y=tx_revenue['Revenue'],
    )
]
plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Revenue',
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [5]:
# sử dụng pct_change() tính phần trăm thay đổi giữa các hàng
tx_revenue['MonthlyGrowth'] = tx_revenue['Revenue'].pct_change()
#vẽ biểu đồ 
plot_data =[
    go.Scatter(
    x=tx_revenue.query("InvoiceYearMonth < 201112")['InvoiceYearMonth'],
    y=tx_revenue.query("InvoiceYearMonth < 201112")['MonthlyGrowth'],
        # lấy đến tháng 12
    )
]
plot_layout = go.Layout(
    xaxis={"type":"category"},
    title='Montly Growth Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

## Phần trăm doanh thu của tháng 4 giảm đột ngột. có thể khách hàng đã mua ít hơn, hoặc mua sản phẩm rẻ hơn, ....

In [6]:
# tạo dataframe với data của UK
tx_uk = tx_data.query("Country=='United Kingdom'").reset_index(drop=True)
# xóa index cũ

In [7]:
#tạo danh sách khách hàng hoạt động trong tháng, tính số CustomerID khác nhau 
tx_monthly_active= tx_uk.groupby('InvoiceYearMonth')['CustomerID'].nunique().reset_index()
tx_monthly_active

InvoiceYearMonth  CustomerID
0             201012         871
1             201101         684
2             201102         714
3             201103         923
4             201104         817
5             201105         985
6             201106         943
7             201107         899
8             201108         867
9             201109        1177
10            201110        1285
11            201111        1548
12            201112         617

In [8]:
#vẽ biểu đồ cột
plot_data = [
    go.Bar(
        x=tx_monthly_active['InvoiceYearMonth'],
        y=tx_monthly_active['CustomerID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type":"category"},
        title = 'Monthly Active Customers'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

### Vào tháng 4 lượng khách hàng đã giảm từ 923 xuống 817

In [9]:
#tạo data tổng sản phẩm hàng tháng
tx_monthly_sales = tx_uk.groupby('InvoiceYearMonth')['Quantity'].sum().reset_index()
tx_monthly_sales

InvoiceYearMonth  Quantity
0             201012    298101
1             201101    237381
2             201102    225641
3             201103    279843
4             201104    257666
5             201105    306452
6             201106    258522
7             201107    324129
8             201108    319804
9             201109    458490
10            201110    455612
11            201111    642281
12            201112    199907

In [10]:
plot_data = [
    go.Bar(
        x=tx_monthly_sales['InvoiceYearMonth'],
        y=tx_monthly_sales['Quantity'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

### Lượng khách hoạt động hàng tháng ảnh hưởng trực tiếp đến số lượng order. Số lượng sản phẩm bán ra vào tháng 4 giảm từ 279843k xuống 257666k

In [11]:
#Tạo data tính trung bình doanh thu trên mỗi hàng
tx_monthly_order_avg = tx_uk.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()
tx_monthly_order_avg

InvoiceYearMonth    Revenue
0             201012  16.865860
1             201101  13.614680
2             201102  16.093027
3             201103  16.716166
4             201104  15.773380
5             201105  17.713823
6             201106  16.714748
7             201107  15.723497
8             201108  17.315899
9             201109  18.931723
10            201110  16.093582
11            201111  16.312383
12            201112  16.247406

In [12]:
plot_data = [
    go.Bar(
        x=tx_monthly_order_avg['InvoiceYearMonth'],
        y=tx_monthly_order_avg['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Order Average'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

### Vào tháng 4 doanh thu trung bình trên 1 sản phẩm giảm

In [13]:
# tạo data ngày mua đầu tiên và CustomerID
tx_min_purchase = tx_uk.groupby('CustomerID').InvoiceDate.min().reset_index()
tx_min_purchase.columns = ['CustomerID','MinPurchaseDate']
#tạo column MinPurchaseYearMonth theo tháng
tx_min_purchase['MinPurchaseYearMonth'] = tx_min_purchase['MinPurchaseDate'].map(lambda date:100*date.year + date.month)
# join 2 data bằng CustomerID
tx_uk = pd.merge(tx_uk, tx_min_purchase, on='CustomerID')
tx_uk.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  \
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom   
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom   
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom   
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom   
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom   

   InvoiceYearMonth  Revenue     MinPurchaseDate  MinPurchaseYearMonth  
0            201012    15.30 2010-12-01 08:26:00                201012  
1            201012    20.34 2010-12-01 08:26:00                201012  
2            201012    22.00 2010-12-01 08:26:00                201012  
3            201012    20.34 2010-12-01 08:26:00                201012  
4            201012    20.34 2010-12-01 08:26:00                201012

In [14]:
#tạo column UserType mặc định là New, lọc những CustomerID đã tồn tại bằng cách so sánh InvoiceYearMonth và MinPurchaseYearMonth
tx_uk['UserType'] ='New'
tx_uk.loc[tx_uk['InvoiceYearMonth']>tx_uk['MinPurchaseYearMonth'],'UserType']='Existing'
#tính tổng doanh thu hàng tháng bằng cách gộp loại khach hàng và tháng mua
tx_uk_type_revenue = tx_uk.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()
#bỏ data của tháng 12
tx_user_type_revenue = tx_uk_type_revenue.query("InvoiceYearMonth != 201012 and InvoiceYearMonth != 201112" )

plot_data = [
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'Existing'")['InvoiceYearMonth'],
        y=tx_user_type_revenue.query("UserType == 'Existing'")['Revenue'],
        name = 'Existing'
    ),
    go.Scatter(
        x=tx_user_type_revenue.query("UserType == 'New'")['InvoiceYearMonth'],
        y=tx_user_type_revenue.query("UserType == 'New'")['Revenue'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type":"category"},
        title='New vs Existing'
)
fig= go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)



### Khách hàng mới ở đây được định nghĩa là khách hàng mua hàng lần đầu tiên qua mỗi tháng

In [15]:
# tạo data phần trăm khách hàng mới
tx_user_ratio = tx_uk.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique()/tx_uk.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique() 
tx_user_ratio = tx_user_ratio.reset_index()
tx_user_ratio = tx_user_ratio.dropna()
tx_user_ratio

InvoiceYearMonth  CustomerID
1             201101    1.124224
2             201102    0.904000
3             201103    0.792233
4             201104    0.510166
5             201105    0.343793
6             201106    0.281250
7             201107    0.236589
8             201108    0.192572
9             201109    0.304878
10            201110    0.328852
11            201111    0.236422
12            201112    0.058319

### Ratio = Khách hàng mới/ khách hàng cũ mỗi tháng

In [16]:
plot_data =[
    go.Bar(
        x=tx_user_ratio.query("InvoiceYearMonth > 201101 and InvoiceYearMonth < 201112")['InvoiceYearMonth'],
        y=tx_user_ratio.query("InvoiceYearMonth > 201101 and InvoiceYearMonth < 201112")['CustomerID'],
    )
]
plot_layout = go.Layout (
        xaxis={"type": "category"},
        title='New Customer Ratio'
)
fig=go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

### Tháng 4 tỉ lệ khách hàng mới giảm mạnh và tiếp tục giảm về những tháng sau có thể có vấn đề về sản phẩm hoặc dịch vụ

In [17]:
#xác định người dùng hoạt động bằng xác nhận việc chi của họ mỗi tháng
tx_user_purchase = tx_uk.groupby(['CustomerID','InvoiceYearMonth'])['Revenue'].sum().reset_index()
# tạo bảng chéo bằng crosstab thể hiện trạng thái hoạt động của khách hàng hàng tháng
tx_retention = pd.crosstab(tx_user_purchase['CustomerID'],tx_user_purchase['InvoiceYearMonth']).reset_index()
tx_retention.head()


InvoiceYearMonth  CustomerID  201012  201101  201102  201103  201104  201105  \
0                    12346.0       0       1       0       0       0       0   
1                    12747.0       1       1       0       1       0       1   
2                    12748.0       1       1       1       1       1       1   
3                    12749.0       0       0       0       0       0       1   
4                    12820.0       0       1       0       0       0       0   

InvoiceYearMonth  201106  201107  201108  201109  201110  201111  201112  
0                      0       0       0       0       0       0       0  
1                      1       0       1       0       1       1       1  
2                      1       1       1       1       1       1       1  
3                      0       0       1       0       0       1       1  
4                      0       0       0       1       1       0       1

In [18]:
# tạo mảng lưu lượng khách ở lại và tổng số khách mỗi tháng bắt đầu tính từ tháng 2/2011 đén tháng 11
months = tx_retention.columns[2:] # bỏ 201012, 201101
retention_array = [] # khai báo mảng rỗng
for i in range(len(months)-1): # tạo vòng lặp với số lần lặp đến tháng 11 vì star range() = 0
    retention_data = {} # tạo từ điển
    selected_month = months[i+1] #chọn tháng cần tính tổng lượng khách
    prev_month = months[i] 
    # tính lượng khách ở lại mỗi tháng
    retention_data['InvoiceYearMonth'] = int(selected_month) 
    retention_data['TotalUserCount']= tx_retention[selected_month].sum() #tổng khách trong tháng
    retention_data['RetainedUserCount'] =tx_retention[(tx_retention[selected_month]>0) &(tx_retention[prev_month]>0)][selected_month].sum()
    #tính khách hàng ở lại với điều kiện =1
    retention_array.append(retention_data) #thêm data  vào mảng



In [19]:
months

Index([201101, 201102, 201103, 201104, 201105, 201106, 201107, 201108, 201109,
       201110, 201111, 201112],
      dtype='object', name='InvoiceYearMonth')

In [20]:
retention_array

[{'InvoiceYearMonth': 201102, 'TotalUserCount': 714, 'RetainedUserCount': 263},
 {'InvoiceYearMonth': 201103, 'TotalUserCount': 923, 'RetainedUserCount': 305},
 {'InvoiceYearMonth': 201104, 'TotalUserCount': 817, 'RetainedUserCount': 310},
 {'InvoiceYearMonth': 201105, 'TotalUserCount': 985, 'RetainedUserCount': 369},
 {'InvoiceYearMonth': 201106, 'TotalUserCount': 943, 'RetainedUserCount': 417},
 {'InvoiceYearMonth': 201107, 'TotalUserCount': 899, 'RetainedUserCount': 379},
 {'InvoiceYearMonth': 201108, 'TotalUserCount': 867, 'RetainedUserCount': 391},
 {'InvoiceYearMonth': 201109,
  'TotalUserCount': 1177,
  'RetainedUserCount': 417},
 {'InvoiceYearMonth': 201110,
  'TotalUserCount': 1285,
  'RetainedUserCount': 502},
 {'InvoiceYearMonth': 201111,
  'TotalUserCount': 1548,
  'RetainedUserCount': 616},
 {'InvoiceYearMonth': 201112, 'TotalUserCount': 617, 'RetainedUserCount': 402}]

In [21]:
#tạo table từ array
tx_retention = pd.DataFrame(retention_array)

In [22]:
 tx_retention

InvoiceYearMonth  TotalUserCount  RetainedUserCount
0             201102             714                263
1             201103             923                305
2             201104             817                310
3             201105             985                369
4             201106             943                417
5             201107             899                379
6             201108             867                391
7             201109            1177                417
8             201110            1285                502
9             201111            1548                616
10            201112             617                402

In [23]:
tx_retention['RetentionRate']= tx_retention['RetainedUserCount']/tx_retention['TotalUserCount']
#tính per

In [24]:
plot_data = [
    go.Scatter(
        x=tx_retention.query("InvoiceYearMonth<201112")['InvoiceYearMonth'],
        y=tx_retention.query("InvoiceYearMonth<201112")['RetentionRate'],
        name="organic"
    )
    
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Retention Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)


 ### Tỉ lệ giữ chân của tháng 4 tăng so với tháng 3. nhưng tháng 3 tỷ lệ này giảm sâu nên không phản ánh được điều gì nhiều

In [25]:
tx_retention

InvoiceYearMonth  TotalUserCount  RetainedUserCount  RetentionRate
0             201102             714                263       0.368347
1             201103             923                305       0.330444
2             201104             817                310       0.379437
3             201105             985                369       0.374619
4             201106             943                417       0.442206
5             201107             899                379       0.421580
6             201108             867                391       0.450980
7             201109            1177                417       0.354291
8             201110            1285                502       0.390661
9             201111            1548                616       0.397933
10            201112             617                402       0.651540

In [30]:
tx_retention = pd.crosstab(tx_user_purchase['CustomerID'], tx_user_purchase['InvoiceYearMonth']).reset_index()
new_column_names = [ 'm_' + str(column) for column in tx_retention.columns]
#đổi tên các cột
tx_retention.columns = new_column_names


In [31]:
tx_retention

m_CustomerID  m_201012  m_201101  m_201102  m_201103  m_201104  \
0          12346.0         0         1         0         0         0   
1          12747.0         1         1         0         1         0   
2          12748.0         1         1         1         1         1   
3          12749.0         0         0         0         0         0   
4          12820.0         0         1         0         0         0   
...            ...       ...       ...       ...       ...       ...   
3945       18280.0         0         0         0         1         0   
3946       18281.0         0         0         0         0         0   
3947       18282.0         0         0         0         0         0   
3948       18283.0         0         1         1         0         1   
3949       18287.0         0         0         0         0         0   

      m_201105  m_201106  m_201107  m_201108  m_201109  m_201110  m_201111  \
0            0         0         0         0         0         0         0   
1            1         1         0         1         0         1         1   
2            1         1         1         1         1         1         1   
3            1         0         0         1         0         0         1   
4            0         0         0         0         1         1         0   
...        ...       ...       ...       ...       ...       ...       ...   
3945         0         0         0         0         0         0         0   
3946         0         1         0         0         0         0         0   
3947         0         0         0         1         0         0         0   
3948         1         1         1         0         1         1         1   
3949         1         0         0         0         0         1         0   

      m_201112  
0            0  
1            1  
2            1  
3            1  
4            1  
...        ...  
3945         0  
3946         0  
3947         1  
3948         1  
3949         0  

[3950 rows x 14 columns]

In [32]:
months

Index([201101, 201102, 201103, 201104, 201105, 201106, 201107, 201108, 201109,
       201110, 201111, 201112],
      dtype='object', name='InvoiceYearMonth')

In [33]:
retention_array = []
for i in range(len(months)):# i chạy từ 0 -11
    retention_data = {}
    selected_month = months[i]
    prev_months = months[:i] # Tháng được chọn trở về trước
    next_months = months[i+1:]# tháng tiếp theo đổ đi
    for prev_month in prev_months:
        retention_data[prev_month] = np.nan
        
    total_user_count =  retention_data['TotalUserCount'] = tx_retention['m_' + str(selected_month)].sum()
    retention_data[selected_month] = 1 #đặt 100% tháng ban đầu
    
    query = "{} > 0".format('m_' + str(selected_month))
    # đặt điều kiện tồn tại của khách hàng qua mỗi tháng

    for next_month in next_months:
        query = query + " and {} > 0".format(str('m_' + str(next_month)))
        # khách hàng ở lại từ tháng được chọn qua các tháng tiếp theo
        retention_data[next_month] = np.round(tx_retention.query(query)['m_' + str(next_month)].sum()/total_user_count,2)
    retention_array.append(retention_data) # nạp vào mảng

In [35]:
retention_array

[{'TotalUserCount': 684,
  201101: 1,
  201102: 0.38,
  201103: 0.26,
  201104: 0.18,
  201105: 0.15,
  201106: 0.13,
  201107: 0.12,
  201108: 0.11,
  201109: 0.1,
  201110: 0.08,
  201111: 0.08,
  201112: 0.07},
 {201101: nan,
  'TotalUserCount': 714,
  201102: 1,
  201103: 0.43,
  201104: 0.23,
  201105: 0.19,
  201106: 0.16,
  201107: 0.14,
  201108: 0.12,
  201109: 0.11,
  201110: 0.1,
  201111: 0.09,
  201112: 0.07},
 {201101: nan,
  201102: nan,
  'TotalUserCount': 923,
  201103: 1,
  201104: 0.34,
  201105: 0.23,
  201106: 0.17,
  201107: 0.13,
  201108: 0.11,
  201109: 0.11,
  201110: 0.09,
  201111: 0.09,
  201112: 0.06},
 {201101: nan,
  201102: nan,
  201103: nan,
  'TotalUserCount': 817,
  201104: 1,
  201105: 0.45,
  201106: 0.28,
  201107: 0.2,
  201108: 0.16,
  201109: 0.15,
  201110: 0.12,
  201111: 0.11,
  201112: 0.08},
 {201101: nan,
  201102: nan,
  201103: nan,
  201104: nan,
  'TotalUserCount': 985,
  201105: 1,
  201106: 0.42,
  201107: 0.25,
  201108: 0.19,
  2

In [34]:
tx_retention = pd.DataFrame(retention_array)
tx_retention.index = months
tx_retention

TotalUserCount  201101  201102  201103  201104  201105  \
InvoiceYearMonth                                                           
201101                       684     1.0    0.38    0.26    0.18    0.15   
201102                       714     NaN    1.00    0.43    0.23    0.19   
201103                       923     NaN     NaN    1.00    0.34    0.23   
201104                       817     NaN     NaN     NaN    1.00    0.45   
201105                       985     NaN     NaN     NaN     NaN    1.00   
201106                       943     NaN     NaN     NaN     NaN     NaN   
201107                       899     NaN     NaN     NaN     NaN     NaN   
201108                       867     NaN     NaN     NaN     NaN     NaN   
201109                      1177     NaN     NaN     NaN     NaN     NaN   
201110                      1285     NaN     NaN     NaN     NaN     NaN   
201111                      1548     NaN     NaN     NaN     NaN     NaN   
201112                       617     NaN     NaN     NaN     NaN     NaN   

                  201106  201107  201108  201109  201110  201111  201112  
InvoiceYearMonth                                                          
201101              0.13    0.12    0.11    0.10    0.08    0.08    0.07  
201102              0.16    0.14    0.12    0.11    0.10    0.09    0.07  
201103              0.17    0.13    0.11    0.11    0.09    0.09    0.06  
201104              0.28    0.20    0.16    0.15    0.12    0.11    0.08  
201105              0.42    0.25    0.19    0.16    0.13    0.12    0.08  
201106              1.00    0.40    0.25    0.19    0.15    0.13    0.09  
201107               NaN    1.00    0.43    0.27    0.19    0.17    0.11  
201108               NaN     NaN    1.00    0.48    0.28    0.23    0.14  
201109               NaN     NaN     NaN    1.00    0.43    0.29    0.15  
201110               NaN     NaN     NaN     NaN    1.00    0.48    0.19  
201111               NaN     NaN     NaN     NaN     NaN    1.00    0.26  
201112               NaN     NaN     NaN     NaN     NaN     NaN    1.00